In [1]:
import os.path
import sys
import numpy as np
np.set_printoptions(3, linewidth=100, suppress=True)

import psi4 
from psi4 import constants as pc
from helper_ccenergy import *
from helper_cchbar import *
from helper_cclambda import *
from helper_ccpert import *
psi4.set_memory(int(2e9), False)
psi4.core.set_output_file('output.dat', False)

ModuleNotFoundError: No module named 'psi4'

In [26]:
def deltaV(mol2,dl):
    bs=psi4.core.BasisSet.build(mol2)
    mints2= psi4.core.MintsHelper(bs)
    V0=np.asarray(mints2.ao_potential())
    for i in range(mol2.natom()):
        mol2.set_nuclear_charge(i,mol2.charge(i)+dl[i])
    V1f=np.asarray(mints2.ao_potential())
    for i in range(mol2.natom()):
        mol2.set_nuclear_charge(i,mol2.charge(i))
    return (V1f-V0)

In [27]:
mol = psi4.geometry('''
    C  0  0  0
    O  0  0  1.1
    symmetry c1
''')
psi4.set_options({'basis': 'sto-3G',
                 'scf_type': 'PK',
    'd_convergence': 1e-10,
    'e_convergence': 1e-10})

In [39]:
rhf_e, rhf_wfn = psi4.energy('SCF', return_wfn=True)

print('RHF Final Energy                          % 16.10f\n' % rhf_e)

# Calculate Ground State CCSD energy
ccsd = HelperCCEnergy(mol, rhf_e, rhf_wfn, memory=2)
ccsd.compute_energy(e_conv=1e-10, r_conv=1e-10)

CCSDcorr_E = ccsd.ccsd_corr_e
CCSD_E = ccsd.ccsd_e

print('\nFinal CCSD correlation energy:          % 16.15f' % CCSDcorr_E)
print('Total CCSD energy:                      % 16.15f' % CCSD_E)

# Now that we have T1 and T2 amplitudes, we can construct
# the pieces of the similarity transformed hamiltonian (Hbar).
cchbar = HelperCCHbar(ccsd)

# Calculate Lambda amplitudes using Hbar
cclambda = HelperCCLambda(ccsd, cchbar)
cclambda.compute_lambda(r_conv=1e-10)

# frequency of calculation
omega_nm = 589
# conver from nm into hartree

omega_nm = 1e+10                                 #######    Huge omega for static polarizability 
omega = (pc.c * pc.h * 1e9) / (pc.hartree2J * omega_nm)

cart = ['X', 'Y', 'Z']
Mu = {}
ccpert = {}
polar_AB = {}

RHF Final Energy                           -111.2190484561


Initalizing CCSD object...

Starting AO ->  MO transformation...
Size of the ERI tensor is 0.00 GB, 10 basis functions.
Building initial guess...

..initialized CCSD in 0.091 seconds.

CCSD Iteration   0: CCSD correlation = -0.122209355619580   dE =  1.22209E-01   MP2
CCSD Iteration   1: CCSD correlation = -0.117888938822379   dE =  4.32042E-03   DIIS = 0
CCSD Iteration   2: CCSD correlation = -0.124934300216099   dE = -7.04536E-03   DIIS = 1
CCSD Iteration   3: CCSD correlation = -0.124756692785084   dE =  1.77607E-04   DIIS = 2
CCSD Iteration   4: CCSD correlation = -0.125737257358572   dE = -9.80565E-04   DIIS = 3
CCSD Iteration   5: CCSD correlation = -0.125778195670906   dE = -4.09383E-05   DIIS = 4
CCSD Iteration   6: CCSD correlation = -0.125784342114257   dE = -6.14644E-06   DIIS = 5
CCSD Iteration   7: CCSD correlation = -0.125782754340304   dE =  1.58777E-06   DIIS = 6
CCSD Iteration   8: CCSD correlation = -0.12578

In [40]:
dir(np)

['ALLOW_THREADS',
 'AxisError',
 'BUFSIZE',
 'CLIP',
 'ComplexWarning',
 'DataSource',
 'ERR_CALL',
 'ERR_DEFAULT',
 'ERR_IGNORE',
 'ERR_LOG',
 'ERR_PRINT',
 'ERR_RAISE',
 'ERR_WARN',
 'FLOATING_POINT_SUPPORT',
 'FPE_DIVIDEBYZERO',
 'FPE_INVALID',
 'FPE_OVERFLOW',
 'FPE_UNDERFLOW',
 'False_',
 'Inf',
 'Infinity',
 'MAXDIMS',
 'MAY_SHARE_BOUNDS',
 'MAY_SHARE_EXACT',
 'MachAr',
 'ModuleDeprecationWarning',
 'NAN',
 'NINF',
 'NZERO',
 'NaN',
 'PINF',
 'PZERO',
 'PackageLoader',
 'RAISE',
 'RankWarning',
 'SHIFT_DIVIDEBYZERO',
 'SHIFT_INVALID',
 'SHIFT_OVERFLOW',
 'SHIFT_UNDERFLOW',
 'ScalarType',
 'Tester',
 'TooHardError',
 'True_',
 'UFUNC_BUFSIZE_DEFAULT',
 'UFUNC_PYVALS_NAME',
 'VisibleDeprecationWarning',
 'WRAP',
 '_NoValue',
 '__NUMPY_SETUP__',
 '__all__',
 '__builtins__',
 '__cached__',
 '__config__',
 '__doc__',
 '__file__',
 '__git_revision__',
 '__loader__',
 '__mkl_version__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_distributor_init',
 '_globa

In [42]:
# Obtain AO Dipole Matrices From Mints

###### replace WITH ALCHEMICAL PERTURBATION 

dV=deltaV(mol,[1,-1])
dipole_array = np.asarray([dV,np.zeros_like(dV),np.zeros_like(dV)])


for i in range(0, 3):
    string = "MU_" + cart[i]

    # Transform dipole integrals from AO to MO basis
    Mu[string] = np.einsum('uj,vi,uv', ccsd.npC, ccsd.npC,
                           np.asarray(dipole_array[i]))

    # Initializing the perturbation classs corresponding to dipole perturabtion at the given omega
    ccpert[string] = HelperCCPert(string, Mu[string], ccsd, cchbar, cclambda,
                                  omega)

    # Solve X and Y amplitudes corresponding to dipole perturabtion at the given omega
    print('\nsolving right hand perturbed amplitudes for %s\n' % string)
    ccpert[string].solve('right', r_conv=1e-10)

    print('\nsolving left hand perturbed amplitudes for %s\n' % string)
    ccpert[string].solve('left', r_conv=1e-10)

# Please refer to eq. 94 of [Koch:1991:3333] for the general form of linear response functions.
# For electric dipole polarizabilities, A = mu[x/y/z] and B = mu[x/y/z],
# Ex. alpha_xy = <<mu_x;mu_y>>, where mu_x = x and mu_y = y

print("\nComputing <<Mu;Mu> tensor @ %d nm" % omega_nm)

for a in range(0, 3):
    str_a = "MU_" + cart[a]
    for b in range(0, 3):
        str_b = "MU_" + cart[b]
        # Computing the alpha tensor
        polar_AB[3 * a + b] = HelperCCLinresp(cclambda, ccpert[str_a],
                                              ccpert[str_b]).linresp()

# Symmetrizing the tensor
for a in range(0, 3):
    for b in range(0, a + 1):
        ab = 3 * a + b
        ba = 3 * b + a
        if a != b:
            polar_AB[ab] = 0.5 * (polar_AB[ab] + polar_AB[ba])
            polar_AB[ba] = polar_AB[ab]

print(
    '\nCCSD Dipole Polarizability Tensor (Length Gauge) at omega = %8.6lf au, %d nm\n'
    % (omega, omega_nm))
print("\t\t%s\t             %s\t                  %s\n" % (cart[0], cart[1],
                                                           cart[2]))

for a in range(0, 3):
    print(" %s %20.10lf %20.10lf %20.10lf\n" %
          (cart[a], polar_AB[3 * a + 0], polar_AB[3 * a + 1],
           polar_AB[3 * a + 2]))

# Isotropic polarizability
trace = polar_AB[0] + polar_AB[4] + polar_AB[8]
Isotropic_polar = trace / 3.0

print(
    " Isotropic CCSD Dipole Polarizability @ %d nm (Length Gauge): %20.10lf a.u."
    % (omega_nm, Isotropic_polar))


solving right hand perturbed amplitudes for MU_X

CCPERT_MU_X Iteration   0: pseudoresponse = 1.806424489300055   dE = -1.80642E+00 
CCPERT_MU_X Iteration   1: pseudoresponse = 1.892163734062834   dE =  8.57392E-02   DIIS = 0
CCPERT_MU_X Iteration   2: pseudoresponse = 1.958902429130213   dE =  6.67387E-02   DIIS = 1
CCPERT_MU_X Iteration   3: pseudoresponse = 1.976334196307438   dE =  1.74318E-02   DIIS = 2
CCPERT_MU_X Iteration   4: pseudoresponse = 1.985899459497770   dE =  9.56526E-03   DIIS = 3
CCPERT_MU_X Iteration   5: pseudoresponse = 1.987408595217285   dE =  1.50914E-03   DIIS = 4
CCPERT_MU_X Iteration   6: pseudoresponse = 1.987460427708006   dE =  5.18325E-05   DIIS = 5
CCPERT_MU_X Iteration   7: pseudoresponse = 1.987471992435166   dE =  1.15647E-05   DIIS = 6
CCPERT_MU_X Iteration   8: pseudoresponse = 1.987471951779582   dE = -4.06556E-08   DIIS = 7
CCPERT_MU_X Iteration   9: pseudoresponse = 1.987471997631834   dE =  4.58523E-08   DIIS = 7
CCPERT_MU_X Iteration  10: ps